In [1]:
import pandas as pd

from sklearn.preprocessing import MinMaxScaler
from sklearn.svm import SVC

In [2]:
df = pd.read_csv("cumulative.csv")
df = df.drop(columns=["rowid", "kepid", "kepoi_name", "kepler_name", "koi_pdisposition", "koi_score", "koi_tce_delivname"])

df = df.dropna(axis='columns', how="all")
df = df.dropna()
df.head()

,koi_disposition,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,koi_period,koi_period_err1,koi_period_err2,koi_time0bk,koi_time0bk_err1,...,koi_steff_err2,koi_slogg,koi_slogg_err1,koi_slogg_err2,koi_srad,koi_srad_err1,koi_srad_err2,ra,dec,koi_kepmag
0,CONFIRMED,0,0,0,0,9.488036,2.775000e-05,-2.775000e-05,170.538750,0.002160,...,-81.0,4.467,0.064,-0.096,0.927,0.105,-0.061,291.93423,48.141651,15.347
1,CONFIRMED,0,0,0,0,54.418383,2.479000e-04,-2.479000e-04,162.513840,0.003520,...,-81.0,4.467,0.064,-0.096,0.927,0.105,-0.061,291.93423,48.141651,15.347
2,FALSE POSITIVE,0,1,0,0,19.899140,1.494000e-05,-1.494000e-05,175.850252,0.000581,...,-176.0,4.544,0.044,-0.176,0.868,0.233,-0.078,297.00482,48.134129,15.436
3,FALSE POSITIVE,0,1,0,0,1.736952,2.630000e-07,-2.630000e-07,170.307565,0.000115,...,-174.0,4.564,0.053,-0.168,0.791,0.201,-0.067,285.53461,48.285210,15.597
4,CONFIRMED,0,0,0,0,2.525592,3.761000e-06,-3.761000e-06,171.595550,0.001130,...,-211.0,4.438,0.070,-0.210,1.046,0.334,-0.133,288.75488,48.226200,15.509


In [3]:
# Removing space for false positive

# new = df["koi_disposition"] == "FALSE POSITIVE"
# df.loc[new, "koi_disposition"] = "False_Poistive"
# df["koi_disposition"]

## Create Train Test Split

In [4]:
# split dataset

X = df.drop("koi_disposition", axis=1)
Y = df["koi_disposition"]
print(X.shape, Y.shape)

(8744, 40) (8744,)


In [5]:
from sklearn.model_selection import train_test_split

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, random_state=69, stratify=Y)

In [6]:
X_train.head()

,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,koi_period,koi_period_err1,koi_period_err2,koi_time0bk,koi_time0bk_err1,koi_time0bk_err2,...,koi_steff_err2,koi_slogg,koi_slogg_err1,koi_slogg_err2,koi_srad,koi_srad_err1,koi_srad_err2,ra,dec,koi_kepmag
1264,0,0,1,0,26.069251,0.000105,-0.000105,143.12549,0.00342,-0.00342,...,-169.0,4.409,0.148,-0.204,0.910,0.252,-0.155,298.90240,40.319530,14.106
2435,0,0,1,0,3.173194,0.000021,-0.000021,133.38365,0.00508,-0.00508,...,-160.0,4.508,0.105,-0.095,0.784,0.109,-0.098,296.88654,42.655079,14.162
3164,0,1,1,0,47.878243,0.000197,-0.000197,193.20518,0.00288,-0.00288,...,-128.0,4.634,0.049,-0.021,0.624,0.040,-0.055,286.24060,42.670231,14.261
8506,0,1,1,0,14.586220,0.000300,-0.000300,135.27460,0.01770,-0.01770,...,-158.0,4.565,0.031,-0.031,0.819,0.358,-0.062,299.06693,47.895191,14.598
6095,1,0,0,1,4.206013,0.000099,-0.000099,132.27940,0.02110,-0.02110,...,-191.0,4.518,0.095,-0.116,0.790,0.146,-0.097,298.50342,41.080181,15.454


# Scale Data using MinMaxScaler

In [7]:
X_scaler = MinMaxScaler().fit(X_train)

In [8]:
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

# Support Vector Machine

In [10]:
model = SVC(kernel='linear')
model.fit(X_train_scaled, Y_train)

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='linear',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [12]:
print(f"Training Data Score: {model.score(X_train_scaled, Y_train)}")
print(f"Testing Data Score: {model.score(X_test_scaled, Y_test)}")

Training Data Score: 0.843092406221409
Testing Data Score: 0.8398902104300091


# GridSearch 

In [15]:
from sklearn.model_selection import GridSearchCV
param_grid = {'C':[1, 5, 10],
              'gamma': [0.0001, 0.001, 0.01]}
grid = GridSearchCV(model, param_grid, verbose=3)

In [16]:
grid.fit(X_train_scaled, Y_train)

Fitting 5 folds for each of 9 candidates, totalling 45 fits
[CV] C=1, gamma=0.0001 ...............................................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV] ................... C=1, gamma=0.0001, score=0.849, total=   0.5s
[CV] C=1, gamma=0.0001 ...............................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.4s remaining:    0.0s


[CV] ................... C=1, gamma=0.0001, score=0.827, total=   0.5s
[CV] C=1, gamma=0.0001 ...............................................


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.9s remaining:    0.0s


[CV] ................... C=1, gamma=0.0001, score=0.838, total=   0.5s
[CV] C=1, gamma=0.0001 ...............................................
[CV] ................... C=1, gamma=0.0001, score=0.835, total=   0.5s
[CV] C=1, gamma=0.0001 ...............................................
[CV] ................... C=1, gamma=0.0001, score=0.850, total=   0.5s
[CV] C=1, gamma=0.001 ................................................
[CV] .................... C=1, gamma=0.001, score=0.849, total=   0.5s
[CV] C=1, gamma=0.001 ................................................
[CV] .................... C=1, gamma=0.001, score=0.827, total=   0.5s
[CV] C=1, gamma=0.001 ................................................
[CV] .................... C=1, gamma=0.001, score=0.838, total=   0.5s
[CV] C=1, gamma=0.001 ................................................
[CV] .................... C=1, gamma=0.001, score=0.835, total=   0.5s
[CV] C=1, gamma=0.001 ................................................
[CV] .

[Parallel(n_jobs=1)]: Done  45 out of  45 | elapsed:   21.6s finished


GridSearchCV(cv=None, error_score=nan,
             estimator=SVC(C=1.0, break_ties=False, cache_size=200,
                           class_weight=None, coef0=0.0,
                           decision_function_shape='ovr', degree=3,
                           gamma='scale', kernel='linear', max_iter=-1,
                           probability=False, random_state=None, shrinking=True,
                           tol=0.001, verbose=False),
             iid='deprecated', n_jobs=None,
             param_grid={'C': [1, 5, 10], 'gamma': [0.0001, 0.001, 0.01]},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=3)

In [17]:
print(grid.best_params_)
print(grid.best_score_)

{'C': 10, 'gamma': 0.0001}
0.8699321873081433


In [18]:
# Make predictions

predictions = grid.predict(X_test_scaled)

In [19]:
# Calculate classification report 

from sklearn.metrics import classification_report

print(classification_report(Y_test, predictions,
                            target_names=["CONFIRMED", "FALSE POSITIVE", "CANDIDATE"]))

                precision    recall  f1-score   support

     CONFIRMED       0.85      0.62      0.72       529
FALSE POSITIVE       0.73      0.88      0.79       568
     CANDIDATE       0.98      1.00      0.99      1089

      accuracy                           0.88      2186
     macro avg       0.85      0.83      0.83      2186
  weighted avg       0.88      0.88      0.87      2186

